In [1053]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import timedelta

## Database Connection

In [1054]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [1055]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [1056]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction


In [1057]:
dim_fase = pd.read_sql_table('dim_fase_servicio', etl_conn) 
fecha = pd.read_sql_table('dim_fecha', etl_conn)
hora = pd.read_sql_table('dim_hora', etl_conn)
mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
estado_servicio = pd.read_sql_table("mensajeria_estadosservicio",url_db)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

## organizar datos

In [1058]:
servicio = servicio[["id", "fecha_solicitud", "hora_solicitud", "mensajero_id"]]
servicio.head(5)

,id,fecha_solicitud,hora_solicitud,mensajero_id
0,34,2023-10-26,09:46:03,NaN
1,35,2023-10-26,11:18:14,7.0
2,36,2023-10-28,19:21:01,NaN
3,41,2023-11-07,09:46:09,NaN
4,42,2023-11-07,09:46:10,NaN


In [1059]:
estado_servicio.drop(columns=["foto","observaciones","es_prueba","foto_binary"], inplace=True)

In [1060]:
estado_servicio[estado_servicio['estado_id'].isin([1, 2, 3, 4, 5, 6])]['estado_id'].value_counts()

estado_id
2    30249
1    29627
4    27542
5    27335
6     8447
3     5202
Name: count, dtype: int64

In [1061]:
estado_servicio_final = estado_servicio
estado_servicio_final.head(5)

,id,fecha,hora,estado_id,servicio_id
0,1014,2024-01-29,01:13:32,4,226
1,1484,2024-01-30,18:45:12,3,79
2,2829,2024-02-06,11:34:04,5,613
3,1888,2024-02-01,14:50:39,4,376
4,32312,2024-04-06,16:11:21,3,7164


In [1062]:
total_nan = estado_servicio_final.isna().sum()
print(total_nan)

id             0
fecha          0
hora           0
estado_id      0
servicio_id    0
dtype: int64


In [1063]:
# Asegurarnos de que las columnas de fecha y hora son de tipo string
estado_servicio_final['fecha'] = estado_servicio_final['fecha'].astype(str)
estado_servicio_final['hora'] = estado_servicio_final['hora'].astype(str)

# Truncar los milisegundos (decimales) de la columna 'hora' si existen
estado_servicio_final['hora'] = estado_servicio_final['hora'].str.split('.').str[0]

# Convertir las columnas de fecha y hora en una única columna de tipo datetime
estado_servicio_final['datetime'] = pd.to_datetime(estado_servicio_final['fecha'] + ' ' + estado_servicio_final['hora'], errors='coerce')

# Ordenar el DataFrame por 'servicio_id', 'estado_id' y 'datetime' de forma descendente
estado_servicio_final = estado_servicio_final.sort_values(by=['servicio_id', 'estado_id', 'datetime'], ascending=[True, True, False])

# Mostrar el resultado
estado_servicio_final.head()

,id,fecha,hora,estado_id,servicio_id,datetime
301,6,2023-09-19,16:22:18,1,7,2023-09-19 16:22:18
667,117,2023-10-13,17:51:20,2,7,2023-10-13 17:51:20
686,134,2023-10-31,12:02:48,4,7,2023-10-31 12:02:48
687,135,2023-10-31,17:07:55,5,7,2023-10-31 17:07:55
688,136,2023-10-31,12:16:00,6,7,2023-10-31 12:16:00


In [1064]:
print(len(estado_servicio_final))

128402


## Transformation

### Hour Process

In [1065]:
# Extrae solo la hora como un número entero
estado_servicio_final['hora_integer'] = pd.to_datetime(estado_servicio_final['hora'], format='%H:%M:%S').dt.hour
estado_servicio_final = pd.merge(estado_servicio_final, hora[["key_hora"]], left_on="hora_integer", right_on="key_hora", how="left")
estado_servicio_final.head(10)

,id,fecha,hora,estado_id,servicio_id,datetime,hora_integer,key_hora
0,6,2023-09-19,16:22:18,1,7,2023-09-19 16:22:18,16,16
1,117,2023-10-13,17:51:20,2,7,2023-10-13 17:51:20,17,17
2,134,2023-10-31,12:02:48,4,7,2023-10-31 12:02:48,12,12
3,135,2023-10-31,17:07:55,5,7,2023-10-31 17:07:55,17,17
4,136,2023-10-31,12:16:00,6,7,2023-10-31 12:16:00,12,12
5,7,2023-09-19,16:30:05,1,8,2023-09-19 16:30:05,16,16
6,241,2023-12-20,20:14:43,2,8,2023-12-20 20:14:43,20,20
7,5355,2024-02-14,15:34:18,4,8,2024-02-14 15:34:18,15,15
8,34061,2024-04-09,16:08:35,5,8,2024-04-09 16:08:35,16,16
9,8,2023-09-19,16:30:05,1,9,2023-09-19 16:30:05,16,16


### Date Process

In [1066]:
mensajero.rename(columns={'id':'mensajero_id'}, inplace=True)
estado_servicio_final['fecha'] = pd.to_datetime(estado_servicio_final['fecha'], errors='coerce')

estado_servicio_final = pd.merge(
    estado_servicio_final,
    fecha[["date", "key_fecha"]],
    left_on="fecha",
    right_on="date",
    how="left"
)

print(servicio.columns)
print(mensajero.columns)

servicio = pd.merge(servicio, mensajero[["mensajero_id", "key_mensajero"]], left_on="mensajero_id", right_on="mensajero_id", how="left")
servicio.head(5)

Index(['id', 'fecha_solicitud', 'hora_solicitud', 'mensajero_id'], dtype='object')
Index(['key_mensajero', 'mensajero_id', 'activo', 'salario',
       'ciudad_operacion', 'departamento_operacion', 'first_name', 'last_name',
       'email', 'saved'],
      dtype='object')


,id,fecha_solicitud,hora_solicitud,mensajero_id,key_mensajero
0,34,2023-10-26,09:46:03,NaN,NaN
1,35,2023-10-26,11:18:14,7.0,13.0
2,36,2023-10-28,19:21:01,NaN,NaN
3,41,2023-11-07,09:46:09,NaN,NaN
4,42,2023-11-07,09:46:10,NaN,NaN


In [1067]:
estado_servicio_final.head(5)

,id,fecha,hora,estado_id,servicio_id,datetime,hora_integer,key_hora,date,key_fecha
0,6,2023-09-19,16:22:18,1,7,2023-09-19 16:22:18,16,16,2023-09-19,253
1,117,2023-10-13,17:51:20,2,7,2023-10-13 17:51:20,17,17,2023-10-13,277
2,134,2023-10-31,12:02:48,4,7,2023-10-31 12:02:48,12,12,2023-10-31,295
3,135,2023-10-31,17:07:55,5,7,2023-10-31 17:07:55,17,17,2023-10-31,295
4,136,2023-10-31,12:16:00,6,7,2023-10-31 12:16:00,12,12,2023-10-31,295


### Phase Process

In [1068]:
fase_iniciado = estado_servicio_final[estado_servicio_final['estado_id'] == 1]
fase_mensajero_asignado = estado_servicio_final[estado_servicio_final['estado_id'] == 2]
fase_recogido_mensajero = estado_servicio_final[estado_servicio_final['estado_id'] == 4]
fase_entregado_destino = estado_servicio_final[estado_servicio_final['estado_id'] == 5]
fase_terminado = estado_servicio_final[estado_servicio_final['estado_id'] == 6]

In [1069]:
# Ordenar el DataFrame por 'servicio_id', 'estado_id' y 'datetime' de forma descendente
fase_iniciado = fase_iniciado.sort_values(by=['servicio_id', 'estado_id', 'datetime'], ascending=[True, True, True])

In [1070]:
fase_iniciado.head(5)

,id,fecha,hora,estado_id,servicio_id,datetime,hora_integer,key_hora,date,key_fecha
0,6,2023-09-19,16:22:18,1,7,2023-09-19 16:22:18,16,16,2023-09-19,253
5,7,2023-09-19,16:30:05,1,8,2023-09-19 16:30:05,16,16,2023-09-19,253
9,8,2023-09-19,16:30:05,1,9,2023-09-19 16:30:05,16,16,2023-09-19,253
11,9,2023-09-19,16:35:52,1,10,2023-09-19 16:35:52,16,16,2023-09-19,253
15,10,2023-09-19,16:37:54,1,11,2023-09-19 16:37:54,16,16,2023-09-19,253


In [1071]:
fase_iniciado.rename(columns={'fecha':'fecha_inicio', 'hora':'hora_inicio', 'key_fecha':'key_fecha_inicio', 'key_hora':'key_hora_inicio','datetime':'datetime_inicio'}, inplace=True)
fase_iniciado = fase_iniciado[['servicio_id','fecha_inicio','hora_inicio','key_fecha_inicio','estado_id','key_hora_inicio','datetime_inicio']]
fase_iniciado.rename(columns={'servicio_id':'id'}, inplace=True)

fase_mensajero_asignado.rename(columns={'fecha':'fecha_mensajero_asignado', 'hora':'hora_mensajero_asignado', 'key_fecha':'key_fecha_mensajero_asignado','key_hora':'key_hora_mensajero_asignado','datetime':'datetime_mensajero_asignado'}, inplace=True)
fase_mensajero_asignado = fase_mensajero_asignado[['servicio_id','fecha_mensajero_asignado','hora_mensajero_asignado','key_fecha_mensajero_asignado','estado_id','key_hora_mensajero_asignado','datetime_mensajero_asignado']]
fase_mensajero_asignado.rename(columns={'servicio_id':'id'}, inplace=True)

fase_recogido_mensajero.rename(columns={'fecha':'fecha_recogido_mensajero', 'hora':'hora_recogido_mensajero','key_fecha':'key_fecha_recogido_mensajero', 'key_hora':'key_hora_recogido_mensajero','datetime':'datetime_recogido_mensajero'}, inplace=True)
fase_recogido_mensajero = fase_recogido_mensajero[['servicio_id','fecha_recogido_mensajero','hora_recogido_mensajero','key_fecha_recogido_mensajero','estado_id','key_hora_recogido_mensajero','datetime_recogido_mensajero']]
fase_recogido_mensajero.rename(columns={'servicio_id':'id'}, inplace=True)

fase_entregado_destino.rename(columns={'fecha':'fecha_entregado_destino', 'hora':'hora_entregado_destino','key_fecha':'key_fecha_entregado_destino','key_hora':'key_hora_entregado_destino','datetime':'datetime_entregado_destino'}, inplace=True)
fase_entregado_destino = fase_entregado_destino[['servicio_id','fecha_entregado_destino','hora_entregado_destino','key_fecha_entregado_destino','estado_id','key_hora_entregado_destino','datetime_entregado_destino']]
fase_entregado_destino.rename(columns={'servicio_id':'id'}, inplace=True)

fase_terminado.rename(columns={'fecha':'fecha_terminado', 'hora':'hora_terminado','key_fecha':'key_fecha_terminado','key_hora':'key_hora_terminado','datetime':'datetime_terminado'}, inplace=True)
fase_terminado = fase_terminado[['servicio_id','fecha_terminado','hora_terminado','key_fecha_terminado','estado_id','key_hora_terminado','datetime_terminado']]
fase_terminado.rename(columns={'servicio_id':'id'}, inplace=True)

C:\Users\nicol\AppData\Local\Temp\ipykernel_25960\1420190926.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fase_mensajero_asignado.rename(columns={'fecha':'fecha_mensajero_asignado', 'hora':'hora_mensajero_asignado', 'key_fecha':'key_fecha_mensajero_asignado','key_hora':'key_hora_mensajero_asignado','datetime':'datetime_mensajero_asignado'}, inplace=True)
C:\Users\nicol\AppData\Local\Temp\ipykernel_25960\1420190926.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fase_recogido_mensajero.rename(columns={'fecha':'fecha_recogido_mensajero', 'hora':'hora_recogido_mensajero','key_fecha':'key_fecha_recogido_mensajer

In [1072]:
fase_iniciado.head(5)

,id,fecha_inicio,hora_inicio,key_fecha_inicio,estado_id,key_hora_inicio,datetime_inicio
0,7,2023-09-19,16:22:18,253,1,16,2023-09-19 16:22:18
5,8,2023-09-19,16:30:05,253,1,16,2023-09-19 16:30:05
9,9,2023-09-19,16:30:05,253,1,16,2023-09-19 16:30:05
11,10,2023-09-19,16:35:52,253,1,16,2023-09-19 16:35:52
15,11,2023-09-19,16:37:54,253,1,16,2023-09-19 16:37:54


## Merge de datos

In [1073]:

servicio = pd.merge(servicio, fase_iniciado[['id', 'fecha_inicio', 'hora_inicio', 'key_fecha_inicio', 'key_hora_inicio','datetime_inicio']], left_on="id", right_on="id", how='left')

servicio = pd.merge(servicio, fase_mensajero_asignado[['id','fecha_mensajero_asignado','hora_mensajero_asignado','key_fecha_mensajero_asignado','key_hora_mensajero_asignado','datetime_mensajero_asignado']], left_on="id", right_on="id", how="left")

servicio = pd.merge(servicio, fase_recogido_mensajero[['id','fecha_recogido_mensajero','hora_recogido_mensajero','key_fecha_recogido_mensajero', 'key_hora_recogido_mensajero','datetime_recogido_mensajero']], left_on="id", right_on="id", how="left")

servicio = pd.merge(servicio, fase_entregado_destino[['id','fecha_entregado_destino','hora_entregado_destino','key_fecha_entregado_destino','key_hora_entregado_destino','datetime_entregado_destino']], left_on="id", right_on="id", how="left")

servicio = pd.merge(servicio, fase_terminado[['id','fecha_terminado','hora_terminado','key_fecha_terminado','key_hora_terminado','datetime_terminado']], left_on="id", right_on="id", how="left")


In [1074]:
servicio.head(5)

,id,fecha_solicitud,hora_solicitud,mensajero_id,key_mensajero,fecha_inicio,hora_inicio,key_fecha_inicio,key_hora_inicio,datetime_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,key_hora_mensajero_asignado,datetime_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,key_hora_recogido_mensajero,datetime_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,key_hora_entregado_destino,datetime_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,key_hora_terminado,datetime_terminado
0,34,2023-10-26,09:46:03,NaN,NaN,2023-10-26,09:46:03,290.0,9.0,2023-10-26 09:46:03,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT
1,35,2023-10-26,11:18:14,7.0,13.0,2023-10-26,11:18:14,290.0,11.0,2023-10-26 11:18:14,2023-10-28,14:43:08,292.0,14.0,2023-10-28 14:43:08,2023-10-28,19:45:18,292.0,19.0,2023-10-28 19:45:18,2023-12-07,01:25:34,332.0,1.0,2023-12-07 01:25:34,NaT,NaN,NaN,NaN,NaT
2,36,2023-10-28,19:21:01,NaN,NaN,2023-10-28,19:21:01,292.0,19.0,2023-10-28 19:21:01,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT
3,41,2023-11-07,09:46:09,NaN,NaN,2023-11-07,09:46:09,302.0,9.0,2023-11-07 09:46:09,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT
4,42,2023-11-07,09:46:10,NaN,NaN,2023-11-07,09:46:10,302.0,9.0,2023-11-07 09:46:10,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT


## Tomar un registro por servicio

In [1075]:
servicio_unique = servicio.groupby(['id'], as_index=False).first()
servicio_unique.head(5)

,id,fecha_solicitud,hora_solicitud,mensajero_id,key_mensajero,fecha_inicio,hora_inicio,key_fecha_inicio,key_hora_inicio,datetime_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,key_hora_mensajero_asignado,datetime_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,key_hora_recogido_mensajero,datetime_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,key_hora_entregado_destino,datetime_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,key_hora_terminado,datetime_terminado
0,7,2023-09-19,16:22:18,1.0,0.0,2023-09-19,16:22:18,253.0,16.0,2023-09-19 16:22:18,2023-10-13,17:51:20,277.0,17.0,2023-10-13 17:51:20,2023-10-31,12:02:48,295.0,12.0,2023-10-31 12:02:48,2023-10-31,17:07:55,295.0,17.0,2023-10-31 17:07:55,2023-10-31,12:16:00,295.0,12.0,2023-10-31 12:16:00
1,8,2023-09-19,16:30:05,7.0,13.0,2023-09-19,16:30:05,253.0,16.0,2023-09-19 16:30:05,2023-12-20,20:14:43,345.0,20.0,2023-12-20 20:14:43,2024-02-14,15:34:18,401.0,15.0,2024-02-14 15:34:18,2024-04-09,16:08:35,456.0,16.0,2024-04-09 16:08:35,NaT,None,NaN,NaN,NaT
2,9,2023-09-19,16:30:05,7.0,13.0,2023-09-19,16:30:05,253.0,16.0,2023-09-19 16:30:05,2023-12-28,19:33:01,353.0,19.0,2023-12-28 19:33:01,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT
3,10,2023-09-19,16:35:52,7.0,13.0,2023-09-19,16:35:52,253.0,16.0,2023-09-19 16:35:52,2023-12-28,19:33:07,353.0,19.0,2023-12-28 19:33:07,2024-02-18,00:21:47,405.0,0.0,2024-02-18 00:21:47,2024-03-10,09:58:27,426.0,9.0,2024-03-10 09:58:27,NaT,None,NaN,NaN,NaT
4,11,2023-09-19,16:37:54,7.0,13.0,2023-09-19,16:37:54,253.0,16.0,2023-09-19 16:37:54,2023-12-09,13:13:59,334.0,13.0,2023-12-09 13:13:59,2024-01-31,10:29:55,387.0,10.0,2024-01-31 10:29:55,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT


In [1076]:
# Ajustar el número de filas y columnas a mostrar
pd.set_option('display.max_rows', None)  # Muestra todas las filas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

servicio_id_7 = servicio_unique.loc[servicio_unique['id'] == 26]
servicio_id_7.head(1)


,id,fecha_solicitud,hora_solicitud,mensajero_id,key_mensajero,fecha_inicio,hora_inicio,key_fecha_inicio,key_hora_inicio,datetime_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,key_hora_mensajero_asignado,datetime_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,key_hora_recogido_mensajero,datetime_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,key_hora_entregado_destino,datetime_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,key_hora_terminado,datetime_terminado
19,26,2023-09-23,19:25:56,7.0,13.0,2023-09-23,19:25:56,257.0,19.0,2023-09-23 19:25:56,2023-10-07,16:30:09,271.0,16.0,2023-10-07 16:30:09,2023-10-07,16:30:11,271.0,16.0,2023-10-07 16:30:11,2023-10-07,16:30:21,271.0,16.0,2023-10-07 16:30:21,NaT,None,NaN,NaN,NaT


In [1077]:

len(servicio_unique)

28430

### Calculo días

In [1078]:
servicio_unique['dias_mensajero_asignado'] = (servicio_unique['datetime_mensajero_asignado'] - servicio_unique['datetime_inicio']).dt.days.fillna(0)
servicio_unique['dias_recogido_mensajero'] = (servicio_unique['datetime_recogido_mensajero'] - servicio_unique['datetime_mensajero_asignado']).dt.days.fillna(0)
servicio_unique['dias_entregado_destino'] = (servicio_unique['datetime_entregado_destino'] - servicio_unique['datetime_recogido_mensajero']).dt.days.fillna(0)
servicio_unique['dias_terminado'] = (servicio_unique['datetime_terminado'] - servicio_unique['datetime_entregado_destino']).dt.days.fillna(0)
# Calcular el promedio general de entrega en días y redondear a 2 decimales
servicio_unique['promedio_entrega_general_dias'] = (
    (servicio_unique['dias_mensajero_asignado'] + 
     servicio_unique['dias_recogido_mensajero'] + 
     servicio_unique['dias_entregado_destino'] + 
     servicio_unique['dias_terminado']) / 4
).round(2)
servicio_unique.head(5)

,id,fecha_solicitud,hora_solicitud,mensajero_id,key_mensajero,fecha_inicio,hora_inicio,key_fecha_inicio,key_hora_inicio,datetime_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,key_hora_mensajero_asignado,datetime_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,key_hora_recogido_mensajero,datetime_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,key_hora_entregado_destino,datetime_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,key_hora_terminado,datetime_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general_dias
0,7,2023-09-19,16:22:18,1.0,0.0,2023-09-19,16:22:18,253.0,16.0,2023-09-19 16:22:18,2023-10-13,17:51:20,277.0,17.0,2023-10-13 17:51:20,2023-10-31,12:02:48,295.0,12.0,2023-10-31 12:02:48,2023-10-31,17:07:55,295.0,17.0,2023-10-31 17:07:55,2023-10-31,12:16:00,295.0,12.0,2023-10-31 12:16:00,24.0,17.0,0.0,-1.0,10.0
1,8,2023-09-19,16:30:05,7.0,13.0,2023-09-19,16:30:05,253.0,16.0,2023-09-19 16:30:05,2023-12-20,20:14:43,345.0,20.0,2023-12-20 20:14:43,2024-02-14,15:34:18,401.0,15.0,2024-02-14 15:34:18,2024-04-09,16:08:35,456.0,16.0,2024-04-09 16:08:35,NaT,None,NaN,NaN,NaT,92.0,55.0,55.0,0.0,50.5
2,9,2023-09-19,16:30:05,7.0,13.0,2023-09-19,16:30:05,253.0,16.0,2023-09-19 16:30:05,2023-12-28,19:33:01,353.0,19.0,2023-12-28 19:33:01,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,100.0,0.0,0.0,0.0,25.0
3,10,2023-09-19,16:35:52,7.0,13.0,2023-09-19,16:35:52,253.0,16.0,2023-09-19 16:35:52,2023-12-28,19:33:07,353.0,19.0,2023-12-28 19:33:07,2024-02-18,00:21:47,405.0,0.0,2024-02-18 00:21:47,2024-03-10,09:58:27,426.0,9.0,2024-03-10 09:58:27,NaT,None,NaN,NaN,NaT,100.0,51.0,21.0,0.0,43.0
4,11,2023-09-19,16:37:54,7.0,13.0,2023-09-19,16:37:54,253.0,16.0,2023-09-19 16:37:54,2023-12-09,13:13:59,334.0,13.0,2023-12-09 13:13:59,2024-01-31,10:29:55,387.0,10.0,2024-01-31 10:29:55,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,80.0,52.0,0.0,0.0,33.0


### Cálculo Hora

In [1079]:
# Calcula la diferencia en tiempo
servicio_unique['horas_mensajero_asignado'] = \
    (servicio_unique['datetime_mensajero_asignado'] - servicio_unique['datetime_inicio']).dt.components['hours'] + \
    (servicio_unique['datetime_mensajero_asignado'] - servicio_unique['datetime_inicio']).dt.components['minutes'] / 60
servicio_unique['horas_mensajero_asignado'] = servicio_unique['horas_mensajero_asignado'].fillna(0).round(2).abs()

servicio_unique['horas_recogido_mensajero'] = \
    (servicio_unique['datetime_recogido_mensajero'] - servicio_unique['datetime_mensajero_asignado']).dt.components['hours'] + \
    (servicio_unique['datetime_recogido_mensajero'] - servicio_unique['datetime_mensajero_asignado']).dt.components['minutes'] / 60
servicio_unique['horas_recogido_mensajero'] = servicio_unique['horas_recogido_mensajero'].fillna(0).round(2).abs()

servicio_unique['horas_entregado_destino'] = \
    (servicio_unique['datetime_entregado_destino'] - servicio_unique['datetime_recogido_mensajero']).dt.components['hours'] + \
    (servicio_unique['datetime_entregado_destino'] - servicio_unique['datetime_recogido_mensajero']).dt.components['minutes'] / 60
servicio_unique['horas_entregado_destino'] = servicio_unique['horas_entregado_destino'].fillna(0).round(2).abs()

servicio_unique['horas_terminado'] = \
    (servicio_unique['datetime_terminado'] - servicio_unique['datetime_entregado_destino']).dt.components['hours'] +\
    (servicio_unique['datetime_terminado'] - servicio_unique['datetime_entregado_destino']).dt.components['minutes'] / 60
servicio_unique['horas_terminado'] = servicio_unique['horas_terminado'].fillna(0).round(2).abs()

# Calcular el promedio general de entrega en horas y redondear a 2 decimales
servicio_unique['promedio_entrega_general_horas'] = (
    (servicio_unique['horas_mensajero_asignado'] + 
     servicio_unique['horas_recogido_mensajero'] + 
     servicio_unique['horas_entregado_destino'] + 
     servicio_unique['horas_terminado']) / 4
).round(2)

servicio_unique.head(5)

,id,fecha_solicitud,hora_solicitud,mensajero_id,key_mensajero,fecha_inicio,hora_inicio,key_fecha_inicio,key_hora_inicio,datetime_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,key_hora_mensajero_asignado,datetime_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,key_hora_recogido_mensajero,datetime_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,key_hora_entregado_destino,datetime_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,key_hora_terminado,datetime_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general_dias,horas_mensajero_asignado,horas_recogido_mensajero,horas_entregado_destino,horas_terminado,promedio_entrega_general_horas
0,7,2023-09-19,16:22:18,1.0,0.0,2023-09-19,16:22:18,253.0,16.0,2023-09-19 16:22:18,2023-10-13,17:51:20,277.0,17.0,2023-10-13 17:51:20,2023-10-31,12:02:48,295.0,12.0,2023-10-31 12:02:48,2023-10-31,17:07:55,295.0,17.0,2023-10-31 17:07:55,2023-10-31,12:16:00,295.0,12.0,2023-10-31 12:16:00,24.0,17.0,0.0,-1.0,10.0,1.48,18.18,5.08,19.13,10.97
1,8,2023-09-19,16:30:05,7.0,13.0,2023-09-19,16:30:05,253.0,16.0,2023-09-19 16:30:05,2023-12-20,20:14:43,345.0,20.0,2023-12-20 20:14:43,2024-02-14,15:34:18,401.0,15.0,2024-02-14 15:34:18,2024-04-09,16:08:35,456.0,16.0,2024-04-09 16:08:35,NaT,None,NaN,NaN,NaT,92.0,55.0,55.0,0.0,50.5,3.73,19.32,0.57,0.00,5.90
2,9,2023-09-19,16:30:05,7.0,13.0,2023-09-19,16:30:05,253.0,16.0,2023-09-19 16:30:05,2023-12-28,19:33:01,353.0,19.0,2023-12-28 19:33:01,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,100.0,0.0,0.0,0.0,25.0,3.03,0.00,0.00,0.00,0.76
3,10,2023-09-19,16:35:52,7.0,13.0,2023-09-19,16:35:52,253.0,16.0,2023-09-19 16:35:52,2023-12-28,19:33:07,353.0,19.0,2023-12-28 19:33:07,2024-02-18,00:21:47,405.0,0.0,2024-02-18 00:21:47,2024-03-10,09:58:27,426.0,9.0,2024-03-10 09:58:27,NaT,None,NaN,NaN,NaT,100.0,51.0,21.0,0.0,43.0,2.95,4.80,9.60,0.00,4.34
4,11,2023-09-19,16:37:54,7.0,13.0,2023-09-19,16:37:54,253.0,16.0,2023-09-19 16:37:54,2023-12-09,13:13:59,334.0,13.0,2023-12-09 13:13:59,2024-01-31,10:29:55,387.0,10.0,2024-01-31 10:29:55,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,80.0,52.0,0.0,0.0,33.0,20.60,21.25,0.00,0.00,10.46


## Rellenar datos faltantes

In [1080]:
servicio_unique.head(5)

,id,fecha_solicitud,hora_solicitud,mensajero_id,key_mensajero,fecha_inicio,hora_inicio,key_fecha_inicio,key_hora_inicio,datetime_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,key_hora_mensajero_asignado,datetime_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,key_hora_recogido_mensajero,datetime_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,key_hora_entregado_destino,datetime_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,key_hora_terminado,datetime_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general_dias,horas_mensajero_asignado,horas_recogido_mensajero,horas_entregado_destino,horas_terminado,promedio_entrega_general_horas
0,7,2023-09-19,16:22:18,1.0,0.0,2023-09-19,16:22:18,253.0,16.0,2023-09-19 16:22:18,2023-10-13,17:51:20,277.0,17.0,2023-10-13 17:51:20,2023-10-31,12:02:48,295.0,12.0,2023-10-31 12:02:48,2023-10-31,17:07:55,295.0,17.0,2023-10-31 17:07:55,2023-10-31,12:16:00,295.0,12.0,2023-10-31 12:16:00,24.0,17.0,0.0,-1.0,10.0,1.48,18.18,5.08,19.13,10.97
1,8,2023-09-19,16:30:05,7.0,13.0,2023-09-19,16:30:05,253.0,16.0,2023-09-19 16:30:05,2023-12-20,20:14:43,345.0,20.0,2023-12-20 20:14:43,2024-02-14,15:34:18,401.0,15.0,2024-02-14 15:34:18,2024-04-09,16:08:35,456.0,16.0,2024-04-09 16:08:35,NaT,None,NaN,NaN,NaT,92.0,55.0,55.0,0.0,50.5,3.73,19.32,0.57,0.00,5.90
2,9,2023-09-19,16:30:05,7.0,13.0,2023-09-19,16:30:05,253.0,16.0,2023-09-19 16:30:05,2023-12-28,19:33:01,353.0,19.0,2023-12-28 19:33:01,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,100.0,0.0,0.0,0.0,25.0,3.03,0.00,0.00,0.00,0.76
3,10,2023-09-19,16:35:52,7.0,13.0,2023-09-19,16:35:52,253.0,16.0,2023-09-19 16:35:52,2023-12-28,19:33:07,353.0,19.0,2023-12-28 19:33:07,2024-02-18,00:21:47,405.0,0.0,2024-02-18 00:21:47,2024-03-10,09:58:27,426.0,9.0,2024-03-10 09:58:27,NaT,None,NaN,NaN,NaT,100.0,51.0,21.0,0.0,43.0,2.95,4.80,9.60,0.00,4.34
4,11,2023-09-19,16:37:54,7.0,13.0,2023-09-19,16:37:54,253.0,16.0,2023-09-19 16:37:54,2023-12-09,13:13:59,334.0,13.0,2023-12-09 13:13:59,2024-01-31,10:29:55,387.0,10.0,2024-01-31 10:29:55,NaT,None,NaN,NaN,NaT,NaT,None,NaN,NaN,NaT,80.0,52.0,0.0,0.0,33.0,20.60,21.25,0.00,0.00,10.46


## Eliminar columnas no necesarias

In [1081]:
servicio_unique.drop(columns=[ "fecha_solicitud","hora_solicitud","mensajero_id",
                                "datetime_inicio","datetime_mensajero_asignado","datetime_recogido_mensajero",
                                "datetime_entregado_destino", "datetime_terminado",
                                "fecha_inicio","hora_inicio",
                                "fecha_mensajero_asignado","hora_mensajero_asignado",
                                "fecha_recogido_mensajero", "hora_recogido_mensajero",
                                "fecha_entregado_destino","hora_entregado_destino",
                                "fecha_terminado", "hora_terminado"], inplace=True)
servicio_unique.head(10)

,id,key_mensajero,key_fecha_inicio,key_hora_inicio,key_fecha_mensajero_asignado,key_hora_mensajero_asignado,key_fecha_recogido_mensajero,key_hora_recogido_mensajero,key_fecha_entregado_destino,key_hora_entregado_destino,key_fecha_terminado,key_hora_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general_dias,horas_mensajero_asignado,horas_recogido_mensajero,horas_entregado_destino,horas_terminado,promedio_entrega_general_horas
0,7,0.0,253.0,16.0,277.0,17.0,295.0,12.0,295.0,17.0,295.0,12.0,24.0,17.0,0.0,-1.0,10.00,1.48,18.18,5.08,19.13,10.97
1,8,13.0,253.0,16.0,345.0,20.0,401.0,15.0,456.0,16.0,NaN,NaN,92.0,55.0,55.0,0.0,50.50,3.73,19.32,0.57,0.00,5.90
2,9,13.0,253.0,16.0,353.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.0,0.0,0.0,25.00,3.03,0.00,0.00,0.00,0.76
3,10,13.0,253.0,16.0,353.0,19.0,405.0,0.0,426.0,9.0,NaN,NaN,100.0,51.0,21.0,0.0,43.00,2.95,4.80,9.60,0.00,4.34
4,11,13.0,253.0,16.0,334.0,13.0,387.0,10.0,NaN,NaN,NaN,NaN,80.0,52.0,0.0,0.0,33.00,20.60,21.25,0.00,0.00,10.46
5,12,31.0,253.0,16.0,271.0,13.0,293.0,0.0,293.0,0.0,293.0,0.0,17.0,21.0,0.0,0.0,9.50,20.97,10.88,0.03,0.02,7.98
6,13,13.0,253.0,16.0,271.0,13.0,277.0,23.0,358.0,3.0,358.0,3.0,17.0,6.0,80.0,0.0,25.75,20.27,10.77,3.60,0.00,8.66
7,14,13.0,253.0,16.0,271.0,13.0,409.0,16.0,410.0,12.0,442.0,16.0,17.0,138.0,0.0,32.0,46.75,20.12,2.88,20.15,4.20,11.84
8,15,13.0,253.0,17.0,271.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,17.0,0.0,0.0,0.0,4.25,19.82,0.00,0.00,0.00,4.96
9,16,13.0,253.0,17.0,271.0,12.0,358.0,3.0,358.0,3.0,358.0,3.0,17.0,86.0,0.0,0.0,25.75,19.73,14.73,0.00,0.00,8.62


In [1082]:
len(servicio_unique)

28430

## Load

In [1083]:
servicio_unique.to_sql("hecho_accumulating_snapshot", etl_conn, if_exists="replace", index_label="key_servicio_fase") 

430